
### *`Data Categories`*
1. `Structured Data`
    - Defined Schema and Model.
    - Used by SQL Queries.
    - Ex:- Tables

2. `Un-Structured Data`
    - Didnot fit into Rigid Schema.
    - Difficult to Search or extrach meaning from it.
    - Ex: Images, Text, Audios, Videos,...etc.

---

### *`LLM vs. VLM`*
- `Large Language Model (LLM)`
    - Can Understand the Human Language -> un-structured data (Text) and Give Meaningful Answers.

- `Vision Language Model (VLM)`
    - Can Understand the Vision Data ->  un-structured data (Images, ..) and Give Meaningful Answers.

- **Challenge**:-
    - If you want your LLM to be Trained on your Private Data and gives answers on it, may be 2 approaches will benefit you:-
        1. `Fine-Tuning`:
            - Re-Train LLM on Your Data.
            - Computationally Expensive and Requires a lot of clean, well-formatted training data.
            - Carry the Risk of Damagining Model'Original Knowledge if not done properly -> Leading to Hallucinations.

        2. `Retrieval Augmented Generation (RAG)`:
            - Instead of Re-Training, You feed your private data as context to LLM.
            - LLM uses context to generate accurate, relevant answers.
            - Cheaper, Efficient, Felxible and Safer than Fine-Tuning.

---

- `RAG Context Window Challenge`:
    - Each LLM has Limited Context Window.
    - `Context Window` -->> Max. Amount of Text the Model can see and process at once.

    **Note** If you dump a `huge` document into model's context window `all at once`:
        1. `In-Efficiency` : Model wastes Resources reading a lot of irrelevant information.
        2. `Hallucination Risk`: With too much input -> LLM Struggles to focus on the part of your actual question and give incorrect response.

- We need smarter way to `Retrieve` only the `relevant chunks related to user's query`, `Augment` it in context and ask LLM to `Generate` answer.

---
### *`Semantic Search and Embeddings`*
- Document is splitted into several `chunks`
- `Semantic Search`:
    - Not Keyword Matching.
    - Mathematical Process.
    1. `Embeddings`:
        - Every Chunk is converted to Embedding (Vector that represents the meaning of word in the context of statement)
    2. `Vector Store (Vector DB)`:
        - Special Database to Stores the Embedding in it.
    3. `Searching with Vectors`:
        - When user asks a question, this user query will be converted into `embedding` and then it is used to find the most similar embeddings related to this user query in meaning.
    4. `LLM Generation`:
        - Then Top Relevant Chunks will be passed to LLM to generate the final answer.

---
### `RAG Pipeline`:
1. `Building the Knowledge Base`:
    1. `Document Loading`:
        - May be PDFs, Word Docs or Files from your internal knowledge base.
    2. `Document Splitting`:
        - Break them into smaller, mnageable chunks.
        - Chunks need to be small enough to fit LLM's Context Window.
        - Smaller Chunks imporve Retrieval Accuracy (LLM Focus on Most Relevant Part).
    3. `Embedding Creation`:
        - Each Chunk is converted into Embedding.
        **Note** Why Embeddings ? -> To allow comparsion of chunks by semantic similarity not just keyword matching.
    4. `Storing in Vector Store`:
        - Special DB for Optimized similarity Search to retrieve the most relevant chunks related to user query.
        - All embedings stored in it.

2. `Retrieval + Generation`
    1. `Querying the DB (Retrieval)`:
        - Convert user's query into embedding and compare it against all document embeddings in vector db.
        - Retrieve most relevant chunks related to user query.

    2. `Response Generation`:
        - Pass the Retrieved Chunks + User Quey to LLM.
        - LLM uses these data to generate accurate answers.

![alt text](<RAG Pipeline.png>)

### *`How to create AI RAG app using LangChain to chat with your private data.`*

In [1]:
# %pip install langchain langchain-Groq langchain-community pypdf chromadb

In [14]:
# %pip install transformers sentencepiece faiss-cpu

In [ ]:
# Document Loading
from langchain_community.document_loaders import PyPDFLoader

# Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Convert Chunks into Embeddings
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from langchain.embeddings.base import Embeddings


# Store Embeddings into Vector Store
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = api_key

### *`Knowledge Base`*

#### 1.*`Document Loading`*

In [ ]:
loader = PyPDFLoader("AFCON-English.pdf")

# Load data into Document objects.
# It Creates Document Object for each page
docs = loader.load()

docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2026-01-08T18:41:07+02:00', 'author': 'احمد اكرم كامل عامر', 'moddate': '2026-01-08T18:41:07+02:00', 'source': 'AFCON-English.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='🇬🇧 Comprehensive AFCON Document (2006–2026) \n      1. Winners from 2006 to 2023 \nYear Champion Runner-Up \n2006 Egypt Ivory Coast \n2008 Egypt Cameroon \n2010 Egypt Ghana \n2012 Zambia Ivory Coast \n2013 Nigeria Burkina Faso \n2015 Ivory Coast Ghana \n2017 Cameroon Egypt \n2019 Algeria Senegal \n2021 Senegal Egypt \n2023 Ivory Coast Nigeria \n2025/26 Ongoing Final on Jan 18, 2026 \n \n        2. Key Records \nMost Titles (All-Time): Egypt — 7 titles (including 2006, 2008, 2010). (Ahram Online) \nLongest penalty shootout: Ivory Coast vs Cameroon 12–11 in 2006. (Ahram Online) \nMost goals in a single tournament: 119 goals in AFCON 2023. (Ahram Online) \n \n      

In [4]:
docs[0].metadata

{'producer': 'Microsoft® Word for Microsoft 365',
 'creator': 'Microsoft® Word for Microsoft 365',
 'creationdate': '2026-01-08T18:41:07+02:00',
 'author': 'احمد اكرم كامل عامر',
 'moddate': '2026-01-08T18:41:07+02:00',
 'source': 'AFCON-English.pdf',
 'total_pages': 2,
 'page': 0,
 'page_label': '1'}

In [5]:
docs[0].page_content

'🇬🇧 Comprehensive AFCON Document (2006–2026) \n      1. Winners from 2006 to 2023 \nYear Champion Runner-Up \n2006 Egypt Ivory Coast \n2008 Egypt Cameroon \n2010 Egypt Ghana \n2012 Zambia Ivory Coast \n2013 Nigeria Burkina Faso \n2015 Ivory Coast Ghana \n2017 Cameroon Egypt \n2019 Algeria Senegal \n2021 Senegal Egypt \n2023 Ivory Coast Nigeria \n2025/26 Ongoing Final on Jan 18, 2026 \n \n        2. Key Records \nMost Titles (All-Time): Egypt — 7 titles (including 2006, 2008, 2010). (Ahram Online) \nLongest penalty shootout: Ivory Coast vs Cameroon 12–11 in 2006. (Ahram Online) \nMost goals in a single tournament: 119 goals in AFCON 2023. (Ahram Online) \n \n      3. Top Scorers (AFCON History) \n• Samuel Eto’o (Cameroon): 18 goals (Confédération Africaine de Football) \n• Laurent Pokou (Ivory Coast): 14 goals (Confédération Africaine de Football) \n• Didier Drogba / Hossam Hassan / Patrick Mboma: ~11 goals (Confédération Africaine de Football) \n \n        4. Brief Summary of Each Tour

#### 2.*`Document Splitter`*

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

chunks = text_splitter.split_documents(docs)

chunks

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2026-01-08T18:41:07+02:00', 'author': 'احمد اكرم كامل عامر', 'moddate': '2026-01-08T18:41:07+02:00', 'source': 'AFCON-English.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='🇬🇧 Comprehensive AFCON Document (2006–2026) \n      1. Winners from 2006 to 2023 \nYear Champion Runner-Up \n2006 Egypt Ivory Coast \n2008 Egypt Cameroon \n2010 Egypt Ghana \n2012 Zambia Ivory Coast \n2013 Nigeria Burkina Faso \n2015 Ivory Coast Ghana \n2017 Cameroon Egypt \n2019 Algeria Senegal \n2021 Senegal Egypt \n2023 Ivory Coast Nigeria \n2025/26 Ongoing Final on Jan 18, 2026 \n \n        2. Key Records \nMost Titles (All-Time): Egypt — 7 titles (including 2006, 2008, 2010). (Ahram Online) \nLongest penalty shootout: Ivory Coast vs Cameroon 12–11 in 2006. (Ahram Online) \nMost goals in a single tournament: 119 goals in AFCON 2023. (Ahram Online) \n \n      

In [10]:
print(f"Number of Chunks = {len(chunks)} Chunks")

Number of Chunks = 3 Chunks


In [11]:
chunks[0].metadata

{'producer': 'Microsoft® Word for Microsoft 365',
 'creator': 'Microsoft® Word for Microsoft 365',
 'creationdate': '2026-01-08T18:41:07+02:00',
 'author': 'احمد اكرم كامل عامر',
 'moddate': '2026-01-08T18:41:07+02:00',
 'source': 'AFCON-English.pdf',
 'total_pages': 2,
 'page': 0,
 'page_label': '1'}

In [12]:
chunks[0].page_content

'🇬🇧 Comprehensive AFCON Document (2006–2026) \n      1. Winners from 2006 to 2023 \nYear Champion Runner-Up \n2006 Egypt Ivory Coast \n2008 Egypt Cameroon \n2010 Egypt Ghana \n2012 Zambia Ivory Coast \n2013 Nigeria Burkina Faso \n2015 Ivory Coast Ghana \n2017 Cameroon Egypt \n2019 Algeria Senegal \n2021 Senegal Egypt \n2023 Ivory Coast Nigeria \n2025/26 Ongoing Final on Jan 18, 2026 \n \n        2. Key Records \nMost Titles (All-Time): Egypt — 7 titles (including 2006, 2008, 2010). (Ahram Online) \nLongest penalty shootout: Ivory Coast vs Cameroon 12–11 in 2006. (Ahram Online) \nMost goals in a single tournament: 119 goals in AFCON 2023. (Ahram Online) \n \n      3. Top Scorers (AFCON History) \n• Samuel Eto’o (Cameroon): 18 goals (Confédération Africaine de Football) \n• Laurent Pokou (Ivory Coast): 14 goals (Confédération Africaine de Football) \n• Didier Drogba / Hossam Hassan / Patrick Mboma: ~11 goals (Confédération Africaine de Football) \n \n        4. Brief Summary of Each Tour

#### 3- *`Embeddings, Vector Store and Retrieval`*

In [16]:
class BGEEmbeddings(Embeddings):
    def __init__(self, model_name="BAAI/bge-base-en-v1.5"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()

    def _embed(self, texts):
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():
            outputs = self.model(**inputs)

        # Mean Pooling
        embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        return embeddings.cpu().numpy().tolist()

    def embed_documents(self, texts):
        # For chunks
        texts = ["Represent this passage for retrieval: " + t for t in texts]
        return self._embed(texts)

    def embed_query(self, text):
        # For user query
        text = "Represent this question for searching: " + text
        return self._embed([text])[0]

In [17]:
vector_store = Chroma.from_documents(documents=chunks, 
                    embedding=BGEEmbeddings(),
                    persist_directory="./afcon_chroma_db")

vector_store

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [18]:
retriever = vector_store.as_retriever()

retriever

VectorStoreRetriever(tags=['Chroma', 'BGEEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000011D755174D0>, search_kwargs={})

In [19]:
# Retrieve Most 3 Chunks related to user's query 
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("Who won AFCON 2006?")

result

[Document(metadata={'total_pages': 2, 'page': 0, 'producer': 'Microsoft® Word for Microsoft 365', 'moddate': '2026-01-08T18:41:07+02:00', 'source': 'AFCON-English.pdf', 'author': 'احمد اكرم كامل عامر', 'page_label': '1', 'creationdate': '2026-01-08T18:41:07+02:00', 'creator': 'Microsoft® Word for Microsoft 365'}, page_content='🇬🇧 Comprehensive AFCON Document (2006–2026) \n      1. Winners from 2006 to 2023 \nYear Champion Runner-Up \n2006 Egypt Ivory Coast \n2008 Egypt Cameroon \n2010 Egypt Ghana \n2012 Zambia Ivory Coast \n2013 Nigeria Burkina Faso \n2015 Ivory Coast Ghana \n2017 Cameroon Egypt \n2019 Algeria Senegal \n2021 Senegal Egypt \n2023 Ivory Coast Nigeria \n2025/26 Ongoing Final on Jan 18, 2026 \n \n        2. Key Records \nMost Titles (All-Time): Egypt — 7 titles (including 2006, 2008, 2010). (Ahram Online) \nLongest penalty shootout: Ivory Coast vs Cameroon 12–11 in 2006. (Ahram Online) \nMost goals in a single tournament: 119 goals in AFCON 2023. (Ahram Online) \n \n      

In [20]:
result[2].page_content

'2006: Egypt win in Cairo after penalties. (Wikipedia) \n    2008: Egypt retain title vs Cameroon. (Wikipedia) \n    2010: Egypt complete three consecutive wins. (Wikipedia) \n    2012: Zambia lift historic trophy. (Wikipedia) \n    2013: Nigeria triumphant. (Wikipedia)'

### *`Chatbot`*

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            """
            You are an expert football analyst specialized in the Africa Cup of Nations (AFCON).
            Your task is to answer questions ONLY using the provided retrieved context from the AFCON documents.
            Do NOT use any external knowledge or assumptions.
            If the answer is not explicitly mentioned in the context, respond with:
            "I could not find this information in the provided AFCON documents."

            When answering:
            - Be concise, factual, and accurate
            - Use clear football terminology
            - Reference years, teams, and records exactly as stated
            - If possible, mention the tournament year in your answer

            Your responses must be grounded strictly in the AFCON data.
            """
        ),
        HumanMessagePromptTemplate.from_template(
            """
            Context: {context}

            Question: {question}
            """
        )
    ]
)

llm = ChatGroq(model='openai/gpt-oss-120b')

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000011D0498A3C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000011D0498B0E0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
parser = StrOutputParser()

parser

StrOutputParser()

User Input -> Retriever -> Prompt -> LLM -> Output


- Instead of:

```
output = llm.invoke(
    prompt_template.invoke(
        retriever.invoke(question)
    )
)


In [24]:
# LCEL – LangChain Expression Language
# | : LCEL Pipe Operator
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser())

rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'BGEEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000011D755174D0>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n            You are an expert football analyst specialized in the Africa Cup of Nations (AFCON).\n            Your task is to answer questions ONLY using the provided retrieved context from the AFCON documents.\n            Do NOT use any external knowledge or assumptions.\n            If the answer is not explicitly mentioned in the context, respond with:\n            "I could not find this information in the provided AFCON documents."\n\n            When answering:\n            - Be concise, factual, and accurate\n            - Use clear foot

In [25]:
# Query the system

response = rag_chain.invoke('من فاز بكأس الأمم الإفريقية 2006؟ ومن كان الوصيف؟')

response

'في عام 2006 فاز منتخب **مصر** ببطولة كأس الأمم الإفريقية، وكان **ساحل العاج (إيفوري كوست)** هو الوصيف.'

In [26]:
response = rag_chain.invoke('أي دولة فازت بكأس إفريقيا ثلاث مرات متتالية؟')

response

'مصر هي الدولة التي فازت بكأس إفريقيا ثلاث مرات متتالية (2006، 2008، و2010).'

In [27]:
response = rag_chain.invoke('أي دولة هي الأكثر تتويجاً باللقب من 2006 إلى 2023؟')

response

'الدولة الأكثر تتويجاً باللقب في الفترة من 2006 إلى 2023 هي **مصر**، حيث فازت بالبطولة ثلاث مرات (2006، 2008، 2010) كما هو مذكور في جدول الفائزين والسجلات.'

In [28]:
response = rag_chain.invoke('من فاز بـ AFCON 2012؟ وما كانت قصة فوزه؟')

response

'في عام 2012 فاز **زامبيا** ببطولة أمم أفريقيا، وقد ارتفعت الكأس في “قصة تاريخية” حيث تُوّجت زامبيا باللقب لأول مرة في تاريخها، ما وصفه الملخص بـ«Zambia lift historic trophy».'

In [29]:
response = rag_chain.invoke('What were the most surprising moments in AFCON history since 2006?')

response

'Based on the information in the provided AFCON documents, the following moments have been highlighted as especially notable (and can be seen as surprising) since 2006:\n\n| Year | Moment | Why it stands out (as described in the documents) |\n|------|--------|---------------------------------------------------|\n| 2006 | **Longest penalty shoot‑out – Ivory Coast vs Cameroon (12‑11)** | Recorded as the longest shoot‑out in AFCON history. |\n| 2012 | **Zambia lift a historic trophy** | The document notes Zambia’s “historic trophy,” indicating an unexpected triumph. |\n| 2015 | **Ivory Coast win the final on penalties** | The 2015 final is specifically mentioned as “Ivory Coast triumph in penalties.” |\n| 2017 | **Cameroon win the tournament in Gabon** | Highlighted as “Cameroon win in Gabon,” a notable home‑soil victory. |\n| 2019 | **Algeria become champions** | The 2019 champion is listed as “Algeria champions,” a new winner compared with earlier dominance by Egypt. |\n| 2021 | **Seneg